In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from copy import copy
from pprint import pprint
from plot import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import itertools
import abc
import sys
import re

sys.path.append("../")
import gpusims
import gpusims.plot.metrics as metric
from gpusims.plot.data import PlotData
from gpusims.config import Config, parse_configs
from gpusims.bench import parse_benchmarks

In [3]:
# ocelot data
ocelot_data = {
    # ocelot1: The Design and Implementation Ocelot’s Dynamic Binary Translator from PTX to Multi-Core x86
    "ocelot1": {
        2010: 5,
        2011: 10,
        2012: 2,
        2013: 3,
        2014: 2,
        2015: 2,
        2016: 0,
        2017: 1,
        2018: 0,
        2019: 1,
    },
    # ocelot2: A Characterization and Analysis of PTX Kernels
    "ocelot2": {
        2008: 2,
        2009: 1,
        2010: 17,
        2011: 13,
        2012: 23,
        2013: 17,
        2014: 19,
        2015: 19,
        2016: 14,
        2017: 11,
        2018: 5,
        2019: 12,
        2020: 5,
        2021: 1,
        2022: 4,
    },
    # ocelot3: A Framework for Dynamically Instrumenting GPU Compute Applications within GPU Ocelot
    "ocelot3": {
        2011: 4,
        2012: 12,
        2013: 11,
        2014: 6,
        2015: 14,
        2016: 6,
        2017: 6,
        2018: 6,
        2019: 4,
        2020: 2,
        2021: 6,
    },
    # ocelot4: GPU Application Development, Debugging, and Performance Tuning with GPU Ocelot
    "ocelot4": {
        2011: 1,
        2012: 2,
        2013: 2,
        2014: 0,
        2015: 1,
        2016: 1,
    },
    # ocelot6: Ocelot: A Dynamic Optimization Framework for Bulk-Synchronous Applications in Heterogeneous Systems
    "ocelot6": {
        2009: 1,
        2010: 6,
        2011: 26,
        2012: 53,
        2013: 48,
        2014: 36,
        2015: 36,
        2016: 26,
        2017: 18,
        2018: 15,
        2019: 18,
        2020: 11,
        2021: 10,
        2022: 11,
    },
}
print(sum([sum(v.values()) for v in ocelot_data.values()]))

588


In [4]:
# macsim data
macsim_data = {
    # macsim1: MacSim: A CPU-GPU Heterogeneous Simulation Framework
    "macsim1": {
        2011: 1,
        2012: 0,
        2013: 3,
        2014: 9,
        2015: 6,
        2016: 10,
        2017: 6,
        2018: 9,
        2019: 6,
        2020: 8,
        2021: 8,
        2022: 2,
    }
}
print(sum([sum(v.values()) for v in macsim_data.values()]))

68


In [5]:
# mgpusim data
mgpusim_data = {
    # MGPUSim1: MGPUSim: Enabling Multi-GPU Performance Modeling and Optimization
    "MGPUSim1": {
        2019: 2,
        2020: 26,
        2021: 22,
        2022: 7,
    },
    # MGSimMark1: Mgsim+ mgmark: A framework for multi-gpu system research
    "MGSimMark1": {
        2018: 1,
        2019: 1,
        2020: 7,
        2021: 2,
    },
}
print(sum([sum(v.values()) for v in mgpusim_data.values()]))

68


In [6]:
# barra data
barra_data = {
    # barra1a: Barra, a modular functional gpu simulator for gpgpu
    "barra1a": {
        2009: 1,
        2010: 1,
        2011: 2,
        2012: 1,
        2013: 0,
        2014: 3,
        2015: 1,
        2016: 0,
        2017: 0,
        2018: 0,
        2019: 1
    },
    # barra1: Barra: A parallel functional simulator for gpgpu
    "barra1": {
        2009: 10,
        2010: 4,
        2011: 12,
        2012: 17,
        2013: 23,
        2014: 17,
        2015: 19,
        2016: 18,
        2017: 9,
        2018: 10,
        2019: 12,
        2020: 5,
        2021: 3,
        2022: 3,
    }
}
print(sum([sum(v.values()) for v in barra_data.values()]))

172


In [7]:
# gputejas data
gputejas_data = {
    # gputejas1: GpuTejas: A parallel simulator for GPU architectures
    "gputejas1": {
        2015: 1,
        2016: 1,
        2017: 2,
        2018: 4,
        2019: 6,
        2020: 5,
        2021: 1,
        2022: 5,
    },
}
print(sum([sum(v.values()) for v in gputejas_data.values()]))

25


In [8]:
# multi2sim data
multi2sim_data = {
    # multi2sim1: Multi2Sim: A Simulation Framework for CPU-GPU Computing
    "multi2sim1": {
        2012: 2,
        2013: 24,
        2014: 46,
        2015: 63,
        2016: 80,
        2017: 81,
        2018: 74,
        2019: 65,
        2020: 36,
        2021: 43,
        2022: 22,
    },
    # multi2sim3: Multi2Sim Kepler: A detailed architectural GPU simulator
    "multi2sim3": {
        2017: 2,
        2018: 6,
        2019: 7,
        2020: 8,
        2021: 6,
        2022: 2,
    },
}
print(sum([sum(v.values()) for v in multi2sim_data.values()]))

567


In [9]:
# gpgpusim data
gpgpusim_data = {
    # gpgpusim3: GPGPU-Sim 3. x manual
    "gpgpusim3": {
        2009: 1,
        2010: 0,
        2011: 0,
        2012: 2,
        2013: 5,
        2014: 3,
        2015: 9,
        2016: 7,
        2017: 7,
        2018: 8,
        2019: 8,
        2020: 7,
        2021: 1,
        2022: 1,
    },
    # gpgpusim4: Analyzing CUDA Workloads Using a Detailed GPU Simulator
    "gpgpusim4": {
        2009: 26,
        2010: 39,
        2011: 63,
        2012: 104,
        2013: 159,
        2014: 186,
        2015: 213,
        2016: 186,
        2017: 182,
        2018: 194,
        2019: 159,
        2020: 139,
        2021: 101,
        2022: 61,
    },
    # accelsim1: Accel-Sim: An Extensible Simulation Framework for Validated GPU Modeling
    "accelsim1": {
        2019: 7,
        2020: 15,
        2021: 40,
        2022: 32,
    },
}
print(sum([sum(v.values()) for v in gpgpusim_data.values()]))

1965


In [10]:
other_data = {
    gpusims.TEJAS: gputejas_data,
    "Barra": barra_data,
    "MGPUSim": mgpusim_data,
    gpusims.MACSIM: macsim_data,
}

top_data = {
    "AccelSim": gpgpusim_data,
    "GPU-Ocelot": ocelot_data,
    gpusims.MULTI2SIM: multi2sim_data,
}

NEW_SIM_COLOR = {**SIM_COLOR, **{
    "AccelSim": SIM_COLOR[gpusims.ACCELSIM_PTX],
    "GPU-Ocelot": "#f5c10a",
    "MGPUSim": "#1532ea",
    "Barra": "#23dc2c",
}}

# find min and max year
def find_range(data):
    years = set()
    for sim, papers in data.items():
        for paper, d in papers.items():
            for year in d.keys():
                years.add(year)
    min_year = min(years)
    max_year = max(years)
    return (min_year, max_year)

(min_year, max_year) = find_range({**top_data, **other_data})
print((min_year, max_year))

(2008, 2022)


In [16]:
def dict_to_array(d):
    result = dict()
    for paper, data in d.items():
        for year, count in data.items():
            if year in result:
                result[year] += count
            else:
                result[year] = count
    sorted_data = list(sorted(result.items(), key=lambda x: x[0]))
    return np.array(sorted_data).T

traces = []
subplot_titles = []
for idx, (data, title) in enumerate([(top_data, "Most cited"), (other_data, "Others")]):
    subplot_titles.append(title)
    for paper_idx, (sim, papers) in enumerate(data.items()):
        sim_data = dict_to_array(papers)
        color = NEW_SIM_COLOR[sim]
        # print(sim_data)
        scatter_opts = dict(
            marker = go.scatter.Marker(
                size = 10,
                color = color,
                # color = "rgba(%d, %d, %d, %f)" % (*hex_to_rgb(SIM_COLOR[sim]), 0.7),
                symbol = "x",
            ),
            line = go.scatter.Line(
                color = color,
                width=16,
            )
        )
        traces.append(go.Bar( # go.Scatter(
            x = sim_data[0],
            y = sim_data[1],
            # text = sim_data[1],
            # textposition='auto',
            # textangle=textangle,
            textfont = dict(
                size=20,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            xaxis = "x" if idx == 0 else f"x{idx+1}",
            yaxis = "y" if idx == 0 else f"y{idx+1}",
            name=SIM_NAME.get(sim) or sim,
            showlegend=True,
            # color = NEW_SIM_COLOR[sim], 
            # mode = 'markers',
            # height = 300,
        ))
    
layout = make_subplots(
    rows=1, cols=2,
    subplot_titles=subplot_titles,
    shared_yaxes=False,
    horizontal_spacing=0.1).layout

log = False
ylabel = "Citation count"
xlabel = "Year"
fontsize = 25
title_fontsize = 25
tick_fontsize = 24
legend_fontsize = 28
font_family="Helvetica"

for i in range(1, 3):
    xaxis = "xaxis" if i <= 1 else f"xaxis{i}"
    yaxis = "yaxis" if i <= 1 else f"yaxis{i}"
    if yaxis in layout:
        if log:
            layout[yaxis].type = "log"
        if i <= 1:
            layout[yaxis].title = ylabel
        layout[yaxis].gridcolor = "gray"
        layout[yaxis].zerolinecolor = "gray"
    if xaxis in layout:
        layout[xaxis].title = xlabel
        layout[xaxis].tickmode='linear'
        layout[xaxis].range=[min_year-1, max_year]
        layout[xaxis].tick0=min_year-1
        layout[xaxis].dtick = 3
        layout[xaxis].tickfont = go.layout.xaxis.Tickfont(size = tick_fontsize)
        layout[xaxis].dividerwidth = 0
        layout[xaxis].dividercolor = "white"
# print(layout)

layout.font=go.layout.Font(
    family = font_family,
    color = "black",
    size = fontsize,
)
layout.hoverlabel = dict(
    bgcolor = "white",
    font_size = fontsize,
    font_family = font_family,
)
layout.legend = go.layout.Legend(
    font=go.layout.legend.Font(size = legend_fontsize),
    itemsizing="constant",
)
if False:
    layout.title = dict(
        text="Cumulative citation counts",
        font=go.layout.title.Font(size = fontsize),
        x=0.5,
        y=0.98,
        xanchor="center",
        yanchor="top",
    )
for anno in layout.annotations:
    anno.font = go.layout.annotation.Font(size = title_fontsize)
# print(layout)

layout.barmode = "group"
layout.bargroupgap = 0 # .1
layout.bargap = 0.1

layout.plot_bgcolor = "white"
layout.margin = go.layout.Margin(
    pad=10,
    autoexpand=True,
    l=MARGIN, r=MARGIN, t=MARGIN, b=MARGIN
)
# layout.width = 1900
# layout.height = 850
layout.width = 1400
layout.height = 450
# print(traces)
fig = go.Figure(data=traces, layout=layout)
# fig.show()
filename = Path("./figs") / "citation_counts.pdf"
filename.parent.mkdir(parents=True, exist_ok=True)
fig.write_image(filename, **PDF_OPTS)